In [ ]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

#import data files
mouse_metadata_path = "Resources/Mouse_metadata.csv"
study_results_path = "Resources/Study_results.csv"

In [ ]:
#merge data
